In [1]:
import json
import pandas as pd
import os
import numpy as np

In [2]:
# Mapping for experiments
GENERIC_GREEDY = "2024070401"
GENERIC_LM = "2024070402"

PERSONALIZED_RANDINIT_GREEDY = "2024070302"
PERSONALIZED_RANINIT_LM = "2024070305"

PERSONALIZED_FINETUNED_GREEDY = "2024070301"
PERSONALIZED_FINETUNED_LM = "2024070304"

In [3]:
def load_metrics(metrics_dir: str) -> tuple:

    user_cer_test = []
    user_cer_val = []

    user_metric_files = os.listdir(metrics_dir)

    for user_file in user_metric_files:

        # Loader user results
        user_metrics = json.load(open(os.path.join(metrics_dir, user_file)))

        # Extract CER for val and test splits
        user_cer_test.append(user_metrics['test']['CER'])
        user_cer_val.append(user_metrics['val']['CER'])

    mean_cer_test = np.mean(user_cer_test)
    mean_cer_val = np.mean(user_cer_val)

    return mean_cer_val, mean_cer_test

In [4]:
paper_results = json.load(open('paper_results.json'))

compiled_results = pd.DataFrame(
    
    columns= ['val_greedy_paper', 'val_greedy_ours', 'test_greedy_paper', 'test_greedy_ours', 'val_6_gram_paper', 'val_6_gram_ours', 
              'test_6_gram_paper', 'test_6_gram_ours'],

    index = ['generic', 'personalized_random_init', 'personalized_finetuned']
)

compiled_results.iloc[:, ['paper' in x for x in compiled_results.columns]] = pd.DataFrame(paper_results).T

cer_generic_greedy_val, cer_generic_greedy_test = load_metrics(os.path.join(GENERIC_GREEDY, 'metrics'))
compiled_results.loc['generic', 'val_greedy_ours'] = cer_generic_greedy_val
compiled_results.loc['generic', 'test_greedy_ours'] = cer_generic_greedy_test

cer_generic_lm_val, cer_generic_lm_test = load_metrics(os.path.join(GENERIC_LM, 'metrics'))
compiled_results.loc['generic', 'val_6_gram_ours'] = cer_generic_lm_val
compiled_results.loc['generic', 'test_6_gram_ours'] = cer_generic_lm_test

cer_personalized_randinit_greedy_val, cer_personalized_randinit_greedy_test = load_metrics(os.path.join(PERSONALIZED_RANDINIT_GREEDY, 'metrics'))
compiled_results.loc['personalized_random_init', 'val_greedy_ours'] = cer_personalized_randinit_greedy_val
compiled_results.loc['personalized_random_init', 'test_greedy_ours'] = cer_personalized_randinit_greedy_test

cer_personalized_randinit_lm_val, cer_personalized_randinit_lm_test = load_metrics(os.path.join(PERSONALIZED_RANINIT_LM, 'metrics'))
compiled_results.loc['personalized_random_init', 'val_6_gram_ours'] = cer_personalized_randinit_lm_val
compiled_results.loc['personalized_random_init', 'test_6_gram_ours'] = cer_personalized_randinit_lm_test

cer_personalized_ft_greedy_val, cer_personalized_ft_greedy_test = load_metrics(os.path.join(PERSONALIZED_FINETUNED_GREEDY, 'metrics'))
compiled_results.loc['personalized_finetuned', 'val_greedy_ours'] = cer_personalized_ft_greedy_val
compiled_results.loc['personalized_finetuned', 'test_greedy_ours'] = cer_personalized_ft_greedy_test

cer_personalized_ft_lm_val, cer_personalized_ft_lm_test = load_metrics(os.path.join(PERSONALIZED_FINETUNED_LM, 'metrics'))
compiled_results.loc['personalized_finetuned', 'val_6_gram_ours'] = cer_personalized_ft_lm_val
compiled_results.loc['personalized_finetuned', 'test_6_gram_ours'] = cer_personalized_ft_lm_test

In [5]:
compiled_results.head()

,val_greedy_paper,val_greedy_ours,test_greedy_paper,test_greedy_ours,val_6_gram_paper,val_6_gram_ours,test_6_gram_paper,test_6_gram_ours
generic,55.57,52.656243,55.38,52.268356,52.1,47.55967,51.78,47.049965
personalized_random_init,15.65,18.583681,15.38,17.035794,11.03,12.045632,9.55,9.125092
personalized_finetuned,11.39,13.229574,11.28,13.046594,8.31,8.955916,6.95,7.674819


In [6]:
compiled_results.to_csv('compiled_results.csv')